# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning', 'R. E. Hviding']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

T. Henning  ->  T. Henning  |  ['T. Henning']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
R. v. Boekel  ->  R. v. Boekel  |  ['R. v. Boekel']
T. Henning  ->  T. Henning  |  ['T. Henning']
P. Boley  ->  P. Boley  |  ['P. Boley']
M. Scheuck  ->  M. Scheuck  |  ['M. Scheuck']
R. v. Boekel  ->  R. v. Boekel  |  ['R. v. Boekel']
P. Mollière  ->  P. Mollière  |  ['P. Mollière']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']


Arxiv has 80 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2602.02836
extracting tarball to tmp_2602.02836... done.


Retrieving document from  https://arxiv.org/e-print/2602.03110


extracting tarball to tmp_2602.03110... done.
Retrieving document from  https://arxiv.org/e-print/2602.03393


extracting tarball to tmp_2602.03393... done.


...

P. Boley  ->  P. Boley  |  ['P. Boley']
M. Scheuck  ->  M. Scheuck  |  ['M. Scheuck']
R. v. Boekel  ->  R. v. Boekel  |  ['R. v. Boekel']
Retrieving document from  https://arxiv.org/e-print/2602.03409


/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:745: LatexWarning: Could not extract abstract from tmp_2602.03401/double.tex
  warnings.warn(LatexWarning(f"Could not extract abstract from {self.main_file}"))
/tmp/ipykernel_3068/2822249172.py:52: LatexWarning: 2602.03401 did not run properly
list index out of range
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


extracting tarball to tmp_2602.03409... done.


P. Mollière  ->  P. Mollière  |  ['P. Mollière']


Retrieving document from  https://arxiv.org/e-print/2602.03574
extracting tarball to tmp_2602.03574... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.02836-b31b1b.svg)](https://arxiv.org/abs/2602.02836) | **Unraveling the Brown Dwarf Desert: Four New Discoveries and a Unifying, Period-Coded Picture**  |
|| J. Šubjak, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2026-02-04*|
|*Comments*| *17 pages, 14 figures, submitted to Astronomy & Astrophysics*|
|**Abstract**|            We present four newly validated transiting brown dwarfs identified through TESS photometry and confirmed with high-precision radial velocity measurements obtained from the FEROS and PLATOSpec spectrographs. Notably, three of these companions exhibit orbital periods exceeding 100 days, thereby expanding the sample of long-period transiting brown dwarfs from two to five systems. The host stars of long-period brown dwarfs show mild subsolar metallicity. These discoveries highlight the expansion of the metal-poor, long-period distribution and help us better understand the brown dwarf desert. In our comparative analysis of eccentricity and metallicity demographics, we utilize catalogues of long-period giant planets, brown dwarfs, and low-mass stellar companions. After accounting for tidal influences, the eccentricity distribution aligns with that of low-mass stellar binaries, presenting a different profile than that observed within the giant planet population. Additionally, the metallicity of the host stars reveals a noteworthy trend: short-period transiting brown dwarfs are predominantly associated with metal-rich stars, whereas long-period brown dwarfs are more often found around metal-poor stars, demonstrating statistical similarities to low-mass stellar hosts. This trend has also been previously observed in studies of hot and cold Jupiters and points to a period-coded mixture of channels. A natural explanation is that most brown dwarfs originate from fragmentation at wider separations, with long-period systems retaining this stellar-like imprint, while only those embedded in massive, long-lived, metal-rich protoplanetary discs are efficiently delivered and stabilised to short orbits.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.03393-b31b1b.svg)](https://arxiv.org/abs/2602.03393) | **The asymmetric structure of the inner disc around HD 142527 A with VLTI/MATISSE**  |
|| M. B. Scheuck, et al. -- incl., <mark>R. v. Boekel</mark>, <mark>T. Henning</mark> |
|*Appeared on*| *2026-02-04*|
|*Comments*| **|
|**Abstract**|            Circumstellar discs, and especially their inner regions, covering ranges from <1 au to a few astronomical units, are the birthplaces of terrestrial planets. The inner regions are thought to be similarly diverse in structure as the well-observed outer regions probed by ALMA. Combining data and results from previous studies of the VLTI/PIONIER and VLTI/GRAVITY instruments with new, multi-epoch VLTI/MATISSE observations, we aim to provide a comprehensive picture of the structure of the inner regions of the circumstellar disc around the F-type Herbig Ae/Be star HD 142527 A, the primary of a binary star system. We model the multi-wavelength interferometric data using a parametrised, geometrically thin disc model, allowing for azimuthal asymmetry, exploring a first-order disc modulation and an off-centre Gaussian component. We find time-variable structures in the N-band observables, which we reproduce with time-dependent models. This variability manifests as azimuthally asymmetric emission, evidenced by strong, non-zero closure phases in the N-band data. Fits to individual epochs of the N-band observations yield better $\chi^2_\text{r}$ values than fits to all epochs simultaneously. This suggests substantial changes in the geometry of the inner disc emission from ~1 au up to a few astronomical-unit scales from one year to the next. Moreover, our models produce a very close-in inner disc rim $R_\text{rim}\approx0.1$ au. All together, we find a very complex, substantially non-point symmetric and temporally-variable disc ($r_\text{out}\lesssim6$ au) around the primary. The very close-in inner rim indicates the presence of material inside the typical wall-like sublimation radius $R_\text{rim,literature}\approx0.3$ au. The complex, temporally variable inner-disc geometry is likely affected or even caused by the close passing (~5 au) and short orbit ($P\approx24$ yr) of the companion HD 142527 B.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.03409-b31b1b.svg)](https://arxiv.org/abs/2602.03409) | **The ESO SupJup Survey IX: Isotopic evidence of a recent formation for Luhman 16AB**  |
|| S. d. Regt, et al. -- incl., <mark>P. Mollière</mark> |
|*Appeared on*| *2026-02-04*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**|            The distinct formation pathways of directly-imaged exoplanets and isolated brown dwarfs might leave imprints in the inherited elemental and isotopic abundances, but such measurements require careful characterisation of the atmospheres. In particular, L-T transition objects exhibit signs of dynamics that drive their atmospheres out of chemical equilibrium. In this work, we studied the nearest L-T brown dwarfs, Luhman 16A and B, to assess the chemical dis-equilibrium in their atmospheres and to investigate their elemental and isotopic composition. As part of the ESO SupJup Survey, we obtained high-resolution CRIRES$^+$ K-band spectra of the binary, which were analysed using an atmospheric retrieval framework. We detect and retrieve the abundances of $^{12}$CO, H$_2$O, CH$_4$, NH$_3$, H$_2$S, HF, and the $^{13}$CO isotopologue. Both atmospheres are in chemical dis-equilibrium with somewhat stronger vertical mixing in Luhman 16A compared to B ($K_\mathrm{zz,A}\sim10^{8.7}$, $K_\mathrm{zz,B}\sim10^{8.2}\ \mathrm{cm^2\ s^{-1}}$). The tested chemical models, free- and dis-equilibrium chemistry, yield consistent mixing ratios and agree with earlier work at shorter wavelengths. The gaseous C/O ratios show evidence of oxygen trapping in silicate-oxide clouds. While the C/O ratios are consistent with solar, the metallicities are modestly enhanced with $\mathrm{[C/H]}\sim0.15$. The carbon isotope ratios are measured at $\mathrm{^{12}C/^{13}C_A}=74^{+2}_{-2}$ and $\mathrm{^{12}C/^{13}C_B}=74^{+3}_{-3}$. The coincident constraints of metallicities and isotopes across the binary reinforce their likely shared formation. The $\mathrm{^{12}C/^{13}C}$ ratios are aligned with the present-day interstellar medium, but lower than the solar-system value. This suggests a recent inheritance and corroborates the relatively young age ($\sim500$ Myr) of Luhman 16AB as members of the Oceanus moving group.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.03574-b31b1b.svg)](https://arxiv.org/abs/2602.03574) | **Euclid. Properties and performance of the NISP signal estimator**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2026-02-04*|
|*Comments*| **|
|**Abstract**|            The Euclid spacecraft, located at the second Lagrangian point of the Sun-Earth system, hosts the Near-Infrared Spectrometer and Photometer (NISP) instrument. NISP is equipped with a mosaic of 16 HgCdTe-based detectors to acquire near-infrared photometric and spectroscopic data. To meet the spacecraft's constraints on computational resources and telemetry bandwidth, the near-infrared signal is processed onboard via a dedicated hardware-software architecture designed to fulfil the stringent Euclid's data-quality requirements. A custom application software, running on the two NISP data processing units, implements the NISP signal estimator: an ad-hoc algorithm which delivers accurate flux measurements and simultaneously estimates the quality of signal estimation through the quality factor parameter. This paper investigates the properties of the NISP signal estimator by evaluating its performance during the early flight operations of the NISP instrument. First, we revisit the assumptions on which the inference of the near-infrared signal is based and investigate the origin of the main systematics of the signal estimator through Monte Carlo simulations. Then, we test the flight performance of the NISP signal estimator. Results indicate a systematic bias lower than 0.01 e/s for 99% of the NISP pixel array, well within the noise budget of the estimated signal. We also derive an analytical expression for the variance of the NISP signal estimator, demonstrating its validity, particularly when the covariance matrix is not pre-computed. Finally, we provide a robust statistical framework to interpret the QF parameter, analyse its dependence on the signal estimator bias, and show its sensitivity to cosmic ray hits on NISP detectors. Our findings corroborate previous results on the NISP signal estimator and suggest a leading-order correction based on the agreement between flight data and simulations.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.03110-b31b1b.svg)](https://arxiv.org/abs/2602.03110) | **Evidence for a 3.0$σ$ Deviation in Gravitational Light Deflection from General Relativity at Cosmological Scales with KiDS-Legacy and CMB Lensing**  |
|| G.-H. Du, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2026-02-04*|
|*Comments*| *11 pages, 3 figures*|
|**Abstract**|            General Relativity (GR) faces challenges from cosmic acceleration and observational tensions, necessitating stringent tests at cosmological scales. In this work, we probe GR deviations via a $\mu$-$\Sigma$ modified gravity parameterization, integrating KiDS-Legacy weak lensing (1347 deg$^2$, $z\leq 2.0$), joint CMB data (Planck/ACT/SPT), DESI DR2 BAO, and DES-Dovekie supernovae. KiDS-Legacy significantly improves constraint precision: $\mu_0$ (matter clustering) by $\sim 43\%$ and $\Sigma_0$ (gravitational light deflection) by $\sim 60\%$ relative to CMB alone. In the $\Lambda$CDM background, $\mu_0 = 0.21\pm 0.21$ is consistent with GR, while $\Sigma_0 = 0.149\pm 0.051$ deviates from GR at the 3.0$\sigma$ level -- attributed to large-scale CMB lensing from ACT/SPT. This precise separation of GR-consistent matter clustering and deviant light deflection provides key observational clues for new physics or data systematics. Our work underscores the critical role of synergizing high-precision CMB and WL data in advancing GR tests.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.03401-b31b1b.svg)](https://arxiv.org/abs/2602.03401) | **Unlocking the dynamics of Young Stellar Objects: Time-Domain Interferometry with six 4-m class telescopes**  |
|| A. Soulain, et al. -- incl., <mark>P. Boley</mark>, <mark>M. Scheuck</mark>, <mark>R. v. Boekel</mark> |
|*Appeared on*| *2026-02-04*|
|*Comments*| *3 pages, 2 figures, White paper presented for Expanding Horizons Transforming Astronomy in the 2040s*|
|**Abstract**|            The dynamics of the inner regions of young stellar objects (YSOs) is driven by a variety of physical phenomena, from magnetospheres and accretion to the dust sublimation rim and inner disk flows. These inner environments evolve on timescales of hours to days, exactly when bursts, dips, and rapid structural changes carry the most valuable information about star and planet formations, but remain hardly reachable with current facilities. A better reactive infrastructure with six or more telescopes, combined with alerts from large time-domain surveys (e.g., at the era of LSST/Rubin type facilities), and equipped with instruments spanning from the V-band to the thermal infrared (N), would provide the instantaneous uv-coverage and spectral diagnostics needed to unambiguously interpret and image these events as they happen. Such a world's first time-domain interferometric observatory would enable qualitatively new science: directly linking optical and infrared variability to spatially resolved changes in magnetospheric accretion, inner-disk geometry, and dust and gas dynamics in the innermost astronomical unit. Crucially, connecting these processes to outer-scale unresolved information from JWST, ALMA, and the ELT would yield a complete tomography of the planet-forming region.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2602.02836/./figures/bd_mass_radius_atmo2020_bar03_with4systems_filtered.png', 'tmp_2602.02836/./figures/periodograms.png', 'tmp_2602.02836/./figures/TIC52059926I-cz20210714.png']
copying  tmp_2602.02836/./figures/bd_mass_radius_atmo2020_bar03_with4systems_filtered.png to _build/html/
copying  tmp_2602.02836/./figures/periodograms.png to _build/html/
copying  tmp_2602.02836/./figures/TIC52059926I-cz20210714.png to _build/html/
exported in  _build/html/2602.02836.md
    + _build/html/tmp_2602.02836/./figures/bd_mass_radius_atmo2020_bar03_with4systems_filtered.png
    + _build/html/tmp_2602.02836/./figures/periodograms.png
    + _build/html/tmp_2602.02836/./figures/TIC52059926I-cz20210714.png
found figures ['tmp_2602.03393/./images/overview.png', 'tmp_2602.03393/./images/best_fit.png', 'tmp_2602.03393/./images/nband.png']
copying  tmp_2602.03393/./images/overview.png to _build/html/
copying  tmp_2602.03393/./images/best_fit.png to _build/html/
copying  tmp_2602.03393/

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\bjdtdb}{\ensuremath{\rm{BJD_{TDB}}}}$
$\newcommand{\feh}{\ensuremath{\left[{\rm Fe}/{\rm H}\right]}}$
$\newcommand{\teff}{\ensuremath{T_{\rm eff}}}$
$\newcommand{\ecosw}{\ensuremath{e\cos{\omega_\star}}}$
$\newcommand{\esinw}{\ensuremath{e\sin{\omega_\star}}}$
$\newcommand{\msun}{\ensuremath{ M_\Sun}}$
$\newcommand{\rsun}{\ensuremath{ R_\Sun}}$
$\newcommand{\lsun}{\ensuremath{ L_\Sun}}$
$\newcommand{\mbd}{\ensuremath{ {\rm M_{BD}}}}$
$\newcommand{\mj}{\ensuremath{ {\rm M_{Jup}}}}$
$\newcommand{\rj}{\ensuremath{ {\rm R_{Jup}}}}$
$\newcommand{\me}{\ensuremath{ {\rm M_E}}}$
$\newcommand{\re}{\ensuremath{ {\rm R_E}}}$
$\newcommand{\mst}{\ensuremath{ {\rm M_\odot}}}$
$\newcommand{\rst}{\ensuremath{ {\rm R_\odot}}}$
$\newcommand{\lst}{\ensuremath{ {\rm L_\odot}}}$
$\newcommand{\fave}{\langle F \rangle}$
$\newcommand{\fluxcgs}{10^9 erg s^{-1} cm^{-2}}$
$\newcommand{\arcsec}{^{\prime \prime}}$
$\newcommand{\tess}{\textit{TESS}}$
$\newcommand{\pytransit}{\texttt{PyTransit}}$
$\newcommand{\ldtk}{\texttt{LDTK}}$
$\newcommand{\celerite}{\texttt{Celerite}}$
$\newcommand{\rebound}{\texttt{REBOUND}}$
$\newcommand{\whfast}{\texttt{WHFAST}}$
$\newcommand{\ias}{\texttt{IAS15}}$</div>



<div id="title">

# Unraveling the Brown Dwarf Desert: Four New Discoveries and a Unifying, Period-Coded Picture$\thanks$ 

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2602.02836-b31b1b.svg)](https://arxiv.org/abs/2602.02836)<mark>Appeared on: 2026-02-04</mark> -  _17 pages, 14 figures, submitted to Astronomy & Astrophysics_

</div>
<div id="authors">

J. Šubjak, et al. -- incl., <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** We present four newly validated transiting brown dwarfs identified through _TESS_ photometry and confirmed with high-precision radial velocity measurements obtained from the FEROS and PLATOSpec spectrographs. Notably, three of these companions exhibit orbital periods exceeding 100 days, thereby expanding the sample of long-period transiting brown dwarfs from two to five systems. The host stars of long-period brown dwarfs show mild subsolar metallicity. These discoveries highlight the expansion of the metal-poor, long-period distribution and help us better understand the brown dwarf desert. In our comparative analysis of eccentricity and metallicity demographics, we utilize catalogues of long-period giant planets, brown dwarfs, and low-mass stellar companions. After accounting for tidal influences, the eccentricity distribution aligns with that of low-mass stellar binaries, presenting a different profile than that observed within the giant planet population. Additionally, the metallicity of the host stars reveals a noteworthy trend: short-period transiting brown dwarfs are predominantly associated with metal-rich stars, whereas long-period brown dwarfs are more often found around metal-poor stars, demonstrating statistical similarities to low-mass stellar hosts. This trend has also been previously observed in studies of hot and cold Jupiters and points to a period-coded mixture of channels. A natural explanation is that most brown dwarfs originate from fragmentation at wider separations, with long-period systems retaining this stellar-like imprint, while only those embedded in massive, long-lived, metal-rich protoplanetary discs are efficiently delivered and stabilised to short orbits.

</div>

<div id="div_fig1">

<img src="tmp_2602.02836/./figures/bd_mass_radius_atmo2020_bar03_with4systems_filtered.png" alt="Fig9" width="100%"/>

**Figure 9. -** The population of known transiting brown dwarfs. We filtered out objects with large uncertainties in mass or radius. Star symbols highlight the positions of our systems. The plot is color-coded by orbital period. The mass-radius tracks from ... are utilized. (*fig:mr*)

</div>
<div id="div_fig2">

<img src="tmp_2602.02836/./figures/periodograms.png" alt="Fig12" width="100%"/>

**Figure 12. -** Generalized Lomb-Scargle periodograms of FEROS RVs of stars in our sample from top to bottom: (a) TIC 9344899 RVs, (b) TIC 9344899 RVs minus transiting planet signal, (c) TIC 52059926 RVs, (d) TIC 52059926 RVs minus planet signal, (e) TIC 13344668 RVs, (f) TIC 13344668 RVs minus planet signal. The vertical red lines highlight the orbital period of the companions. Horizontal dashed lines show the theoretical FAP levels of 10\%, 1\%, and 0.1\% for each panel. (*fig:per*)

</div>
<div id="div_fig3">

<img src="tmp_2602.02836/./figures/TIC52059926I-cz20210714.png" alt="Fig1" width="100%"/>

**Figure 1. -** SOAR contrast curve for Cousins $I$ band with a $6$\arcsec$ \times 6$\arcsec$$ reconstructed image of the field. (*fig:speckle_image*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2602.02836"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\acffont}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.25}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.15}$</div>



<div id="title">

# The asymmetric structure of the inner disc around HD 142527 A with $\acs{vlti}$/$\acs{matisse}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2602.03393-b31b1b.svg)](https://arxiv.org/abs/2602.03393)<mark>Appeared on: 2026-02-04</mark> - 

</div>
<div id="authors">

M. B. Scheuck, et al. -- incl., <mark>R. v. Boekel</mark>, <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** Circumstellar discs, and especially their inner regions, covering ranges from $\qty{<1}{\astronomicalunit}$ to a few astronomical units, are the birthplaces of terrestrial planets. The inner regions are thought to be similarly diverse in structure as the well-observed outer regions probed by ALMA. Combining data and results from previous studies of the $\acs{vlti}$ / $\acs{pionier}$ and $\acs{vlti}$ / $\acs{gravity}$ instruments with new, multi-epoch $\acs{vlti}$ / $\acs{matisse}$ observations, we aim to provide a comprehensive picture of the structure of the inner regions of the circumstellar disc around the F-type Herbig Ae/Be star HD 142527 A, the primary of a binary star system. We model the multi-wavelength interferometric data using a parametrised, geometrically thin disc model, allowing for azimuthal asymmetry, exploring a first-order disc modulation and an off-centre Gaussian component. We find time-variable structures in the _N_ -band observables, which we reproduce with time-dependent models.This variability manifests as azimuthally asymmetric emission, evidenced by strong, non-zero closure phases in the _N_ -band data.Fits to individual epochs of the _N_ -band observations yield better $ \chi^2_\text{r} $ values than fits to all epochs simultaneously.This suggests substantial changes in the geometry of the inner disc emission from $\qty{\sim1}{\astronomicalunit}$ up to a few astronomical-unit scales from one year to the next. Moreover, our models produce a very close-in inner disc rim $R_\text{rim}\approx\qty{0.1}{\astronomicalunit}$ .All together, we find a very complex, substantially non-point symmetric and temporally-variable disc ( $r_\text{out}\lesssim\qty{6}{\astronomicalunit}$ ) around the primary. The very close-in inner rim indicates the presence of material inside the typical wall-like sublimation radius $R_\text{rim,literature}\approx\qty{0.3}{\astronomicalunit}$ .The complex, temporally variable inner-disc geometry is likely affected or even caused by the close passing ( $\qty{\sim5}{\astronomicalunit}$ ) and short orbit ( $P\approx\qty{24}{\year}$ ) of the companion HD 142527 B.

</div>

<div id="div_fig1">

<img src="tmp_2602.03393/./images/overview.png" alt="Fig10" width="100%"/>

**Figure 10. -** 
            Overview of binned data used for model-fitting. _Left to right_: the three observables, the total spectrum, the correlated fluxes, and the closure phases. _Top to bottom_: the data from all bands (_H_, _K_, _L_, _M_, and _N_), a zoom-in to the bands of shorter wavelengths (_H_, _K_, _L_, and _M_), and a zoom-in to the _N_ band.
         (*fig:allDataSpatial*)

</div>
<div id="div_fig2">

<img src="tmp_2602.03393/./images/best_fit.png" alt="Fig5" width="100%"/>

**Figure 5. -** 
        Fit to data for the best-fit, one-zone disc model with an off-centre Gaussian asymmetry.
        The data (colored line) is overlaid with the model (black cross). The residuals of the plots are presented in Fig. \ref{fig:bestFitResiduals}.
        _Left_: the total spectrum $F_\nu$. _Middle_: the correlated fluxes $F_{\nu,\text{corr}}$. _Right_: the closure phases $\Phi_{\nu,\text{cp}}$.
        _Top_: the \ac{pionier}(_H_ band), \acs{gravity}(_K_ band), and \ac{matisse}(_L_ and _M_ band) data.
        This data is fitted with all _N_-band data sets and shown, as an example, is the fit with the first _N_-band epoch.
        _Second row to bottom_: the first to third _N_-band epoch (2021, 2022, 2023).
     (*fig:bestFit*)

</div>
<div id="div_fig3">

<img src="tmp_2602.03393/./images/nband.png" alt="Fig7" width="100%"/>

**Figure 7. -** \ac{matisse}_N_-band observations of HD 142527 (Table \ref{tab:observations}). _Left to right_: the baseline distribution, the single dish spectra, the correlated fluxes, and the closure phases.
             (*fig:nbandData*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2602.03393"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\twoentries}[2]{\begin{tabular}{@ r@ }#1\\\big[{\footnotesize #2}\big]\end{tabular}}$
$\newcommand{\arraystretch}{1.3}$</div>



<div id="title">

# The ESO SupJup Survey: IX. Isotopic evidence of a recent formation for Luhman 16AB

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2602.03409-b31b1b.svg)](https://arxiv.org/abs/2602.03409)<mark>Appeared on: 2026-02-04</mark> -  _Accepted for publication in A&A_

</div>
<div id="authors">

S. d. Regt, et al. -- incl., <mark>P. Mollière</mark>

</div>
<div id="abstract">

**Abstract:** The distinct formation pathways proposed for directly imaged exoplanets and isolated brown dwarfs might leave imprints in the inherited chemical composition. Elemental and isotopic tracers could help inform the suspected histories, but this requires a careful characterisation of the sub-stellar atmospheres. In particular, objects at the L-T transition exhibit signs of dynamics that can drive their atmospheres out of chemical equilibrium. In this work, we studied the nearest L-T brown dwarfs, Luhman 16A and B, to assess the chemical disequilibrium in their atmospheres. We also investigated the elemental and isotopic compositions in the context of their probable formation history within the Oceanus moving group. As part of the ESO SupJup Survey, we obtained spatially resolved CRIRES $^+$ K-band spectra of the binary. These high-resolution observations were analysed using an atmospheric retrieval framework that couples the radiative transfer code \texttt{petitRADTRANS} to the \texttt{MultiNest} sampling algorithm. We detect and retrieve the abundances of $^{12}$ CO, $H_2$ O, $CH_4$ , $NH_3$ , $H_2$ S, HF, and the $^{13}$ CO isotopologue. We find that both atmospheres are in chemical disequilibrium with somewhat stronger vertical mixing in Luhman 16A compared to B ( $K_\mathrm{zz,A}\sim$ $10^{8.7}$ vs $K_\mathrm{zz,B}\sim$ $10^{8.2} \mathrm{cm^2 s^{-1}}$ ). The tested chemical models, free-equilibrium and disequilibrium chemistry, yield consistent mixing ratios and agree with earlier work at shorter wavelengths. The free-chemistry gaseous $\mathrm{C/O}$ ratios show evidence of oxygen trapping in silicate-oxide clouds. While the $\mathrm{C/O}$ ratios are consistent with the solar composition, the metallicities are modestly enhanced with $\mathrm{[C/H]}\sim$ $0.15$ . The carbon isotope ratios are measured at $\mathrm{^{12}C/^{13}C_A}=74^{+2}_{-2}$ and $\mathrm{^{12}C/^{13}C_B}=74^{+3}_{-3}$ . The coincident constraints of metallicities and isotopes across the binary provide further evidence in favour of a common formation. The $\mathrm{^{12}C/^{13}C}$ ratios are aligned with the present-day interstellar medium but lower than the Solar System value. This suggests a recent inheritance and corroborates the relatively young age ( $\sim$ $500 \mathrm{Myr}$ ) of Luhman 16A and B as members of the Oceanus moving group.

</div>

<div id="div_fig1">

<img src="tmp_2602.03409/./plots/bestfit_spectra/K_band_spectrum_bestfit_order5.png" alt="Fig6" width="100%"/>

**Figure 6. -** CRIRES$^+$ K-band spectra of Luhman 16A and B in orange and blue, respectively. _Top panel_: Seven spectral orders covered in the K2166 wavelength setting. The telluric absorption is shown as transparent lines. _Lower panels_: Zoom-in of the sixth order. The black observed spectra are overlaid with the best-fitting free-chemistry models in orange and blue. The mean scaled uncertainties are displayed to the right of the residuals in the bottom panel. The fits to the other spectral orders can be found in Appendix \ref{app:best_fitting_spectra}. (*fig:spectrum*)

</div>
<div id="div_fig2">

<img src="tmp_2602.03409/./plots/K_band_VMR_profiles.png" alt="Fig7" width="100%"/>

**Figure 7. -** Comparison of temperature and abundance constraints. _Left panel_: Temperature profiles of Luhman 16A and B, retrieved with the chemical disequilibrium model described in Sect. \ref{sect:meth_chem}. Luhman 16A displays a heating of $\sim$$100 \mathrm{K}$ near the photosphere, which is indicated by the shaded regions in the left side of the panel. _Middle panel_: Mixing and chemical reaction timescales as functions of pressure. The intersection where vertical mixing becomes more efficient than the CO-$CH_4$ reaction is indicated by a horizontal bar. The higher $K_\mathrm{zz}$ for Luhman 16A is visible from its reduced mixing timescale. _Right panels_: Cutouts of the chemical abundance profiles between $0.06$ and $30 \mathrm{bar}$. _From top to bottom_: Chemical disequilibrium and free-chemistry abundances from the K-band analysis of this work, and the CRIRES$^+$ J-band constraints presented in [de Regt, et. al (2025)](https://ui.adsabs.harvard.edu/abs/2025A&A...696A.225D). The envelopes and error bars in all panels show the $68\%$ credible region. (*fig:eqchem_VMRs*)

</div>
<div id="div_fig3">

<img src="tmp_2602.03409/./plots/gaseous_fraction.png" alt="Fig8" width="100%"/>

**Figure 8. -** Abundance profiles of gases as a fraction of the total elemental reservoir of oxygen, carbon, nitrogen, sulphur, and fluorine. The abundances are calculated with \texttt{FastChem Cond} ([Kitzmann, Stock and Patzer 2024](https://ui.adsabs.harvard.edu/abs/2024MNRAS.527.7263K)) , using SONORA Elf Owl temperature structures (shown in the right panel;  ([Mukherjee, et. al 2024](https://ui.adsabs.harvard.edu/abs/2024ApJ...963...73M)) ) that are representative of Y-, T-, L- and late M-type dwarfs. Oxygen and carbon are carried by multiple gaseous molecules ($H_2$O, CO, and $CH_4$), which are shown as their separate and summed profiles. The condensation points of important cloud species are indicated as arrowheads for reference. (*fig:gas_fraction*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2602.03409"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcid}[1]$
$\newcommand{\arraystretch}{1.3}$</div>



<div id="title">

# $\Euclid$: Properties and performance of the NISP signal estimator

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2602.03574-b31b1b.svg)](https://arxiv.org/abs/2602.03574)<mark>Appeared on: 2026-02-04</mark> - 

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** The $\Euclid$ spacecraft, located at the second Lagrangian point of the Sun-Earth system, hosts the Near-Infrared Spectrometer and Photometer (NISP) instrument. NISP is equipped with a mosaic of 16 HgCdTe-based detectors (Teledyne's H2RG) to acquire near-infrared photometric and spectroscopic data. To meet the spacecraft's constraints on computational resources and telemetry bandwidth, the near-infrared signal is processed onboard via a dedicated hardware-software architecture designed to fulfil the stringent _Euclid_ 's data-quality requirements. A custom application software, running on the two NISP data processing units, implements the NISP signal estimator: an ad-hoc algorithm which delivers accurate flux measurements and simultaneously estimates the quality of signal estimation through the quality factor (QF) parameter.  This paper further investigates the properties of the NISP signal estimator by evaluating its performance during the early flight operations of the NISP instrument. First, we revisit the assumptions on which the inference of the near-infrared signal is based and investigate the origin of the main systematics of the signal estimator through Monte Carlo simulations. Then, we test the flight performance of the NISP signal estimator. Results indicate a systematic bias lower than $0.01 {\rm e}^{-} {\rm s}^{-1}$ for $99\%$ of the NISP pixel array, well within the noise budget of the estimated signal. We also derive an analytical expression for the variance of the NISP signal estimator, demonstrating its validity, particularly when the covariance matrix is not pre-computed. Finally, we provide a robust statistical framework to interpret the QF parameter, analyse its dependence on the signal estimator bias, and show its sensitivity to cosmic ray hits on NISP detectors. Our findings corroborate previous results on the NISP signal estimator and suggest a leading-order correction based on the agreement between flight data and simulations.

</div>

<div id="div_fig1">

<img src="tmp_2602.03574/./Figures/bf_fcn_nd11_ng_sR13.png" alt="Fig3.1" width="25%"/><img src="tmp_2602.03574/./Figures/S_MACCspectro_f_bias_fcn_flux.png" alt="Fig3.2" width="25%"/><img src="tmp_2602.03574/./Figures/QF_fcn_nd11_ng_sR13.png" alt="Fig3.3" width="25%"/><img src="tmp_2602.03574/./Figures/QF_MACCspectro_const_bias_fcn_flux.png" alt="Fig3.4" width="25%"/>

**Figure 3. -** *Top panels:*  Bias of the signal estimator. *Bottom panel:* QF values. *Left panels:* Bias as function of $n_{\rm g}$ for $n_{\rm f}=16$, $n_{\rm d}=11$, and $\sigma_R=13 {\rm e}^{-}$ in the case where $f_{\rm 0} > f_{\rm f}$(no folding effect). For $f = f_{\rm 0}$(green squares) for which the off-diagonal entries of the variance matrix are null, the bias is perfectly predicted by the analytical formula (dashed lines) from Eq. (\ref{eq:bias}) for the signal, and Eq. (\ref{eq:QFbias}) for the QF. For $f<f_{\rm 0}$(light green triangles), the theoretical prediction underestimates the bias. For $f>f_{\rm 0}$(dark green dots) the theoretical prediction overestimates the bias. *Right panels:* bias of the signal estimator as a function of flux in spectroscopic readout mode. The flux values $f_{\rm 0}$ are indicated by vertical lines for each of the readout noise values. The $f_{\rm f}=0.1 {\rm e}^{-} {\rm s}^{-1}$ flux value, above which the folding contribution is negligible, is indicated by a grey dash-dot-dotted vertical line. Positive contribution of the $D_{ij}$ to the covariance matrix for $f > f_{\rm 0}$ translates in the measured signal bias lower than the analytical formula, while negative contribution of the $D_{ij}$ translates in the measured signal bias higher than the analytical formula. For $f \gg f_{\rm 0}$ and $f \gg f_{\rm f}$, the bias does not depend on flux. Similar results are obtained in photometric readout mode. (*fig:S_bias_fcn_ng*)

</div>
<div id="div_fig2">

<img src="tmp_2602.03574/./Figures/QF_MCvcDATA_SPECTRO.png" alt="Fig9.1" width="50%"/><img src="tmp_2602.03574/./Figures/QF_MCvcDATA_PHOTO.png" alt="Fig9.2" width="50%"/>

**Figure 9. -** Example (DET 11, SCA 18453) of the performance of the NISP QF from simulated and early flight data. Each distribution reports the QF distribution related to a single photometric or spectroscopic exposure. The grey shaded histograms represent the $\chi^2(n_g-2)$ distribution, corresponding to the expected behaviour of the NISP QF in the absence of anomalies during the signal integration. The red, blue, and green histograms show the QF distributions for three distinct flux regimes: $0.01 {\rm e}^{-} {\rm s}^{-1}$(dark current), $1 {\rm e}^{-} {\rm s}^{-1}$(zodiacal light), and $>10 {\rm e}^{-} {\rm s}^{-1}$(flat field), respectively. Flat field simulations were performed at $20 {\rm e}^{-} {\rm s}^{-1}$ for both NISP channels, while spectroscopic and photometric flight data were acquired at $16 {\rm e}^{-} {\rm s}^{-1}$ and $80 {\rm e}^{-} {\rm s}^{-1}$, respectively. Black dashed vertical lines show the threshold values applied to the NISP QF distributions in the spectroscopic (QF = 50) and photometric (QF = 10) channels.
*Top panels:* Spectroscopic MACC(15,16,11) mode.
*Bottom panels:* Photometric MACC(4,16,4) mode.
*Left panels:* Synthetic QF distributions obtained by processing the simulated data with the per-pixel readout noise map -- labelled as Nominal. At high flux ($20 {\rm e}^{-} {\rm s}^{-1}$, green histogram) the QF distribution perfectly matches the expected $\chi^2_{\nu}$ distribution. As the flux decreases, the QF distributions progressively deviate from the $\chi^2_{\nu}$, as expected from the analysis reported in Sect. \ref{sec:QF}.
*Centre panels:* Synthetic QF distributions obtained by processing the simulated data with the detector-average readout noise value -- labelled as DPA.
At high flux ($20 {\rm e}^{-} {\rm s}^{-1}$, green histogram) the QF distribution closely follows the expected $\chi^2_{\nu}$ distribution, aside from a few outliers. As the flux decreases, the QF distributions progressively deviate from the $\chi^2_{\nu}$ due to the superimposition of the intrinsic bias of the NISP signal estimator and the DPA effect.
*Right panels:*
Early flight data matching MC simulations (DPA case), if space weather effects are accounted for. The NISP QF follows the $\chi^2_{\nu}$ distribution at high flux (green histograms) while increasingly deviates from it as the flux decreases (red and blue diagrams). The outlier tail in the NISP flight data mainly results from the impact of solar energetic particles on the NISP detectors.
 (*fig:QF_MCvsDATA*)

</div>
<div id="div_fig3">

<img src="tmp_2602.03574/./Figures/bf_MACCspectro_DPU_bias_3f.png" alt="Fig6.1" width="25%"/><img src="tmp_2602.03574/./Figures/bf_MACCspectro_DPUfe_bias.png" alt="Fig6.2" width="25%"/><img src="tmp_2602.03574/./Figures/QF_MACCspectro_DPU_bias2.png" alt="Fig6.3" width="25%"/><img src="tmp_2602.03574/./Figures/QF_MACCspectro_DPUfe_bias.png" alt="Fig6.4" width="25%"/>

**Figure 6. -** DPA induced bias to the signal estimator $\hat{g}[\Delta \vec{G}]_{n_{\rm g}}$(top panels) and to the QF (bottom panels). For each value of the pixel simulated readout noise $\sigma_{\sfont R}$, we have chosen the specific flux value $f=f_{\rm0}$ for which the off-diagonal terms in the covariance matrix vanish and the theoretical bias model conforms to the simulated values (thick solid lines). Additionally, for each $\sigma_{\sfont R}$ we perform simulations for $f=\frac{f_{\rm0}}{2}$(dashed lines) and $f=2f_0$(dotted lines). *Left panels:* Bias due to $\Delta \sigma_{\sfont R}$. The shaded areas correspond to the typical \Euclid case of possible values of $-3 < \Delta \sigma_{\sfont R}<6$ as 99\% of pixels are in this range of $\Delta \sigma_{\sfont R}$ -- see Sect. \ref{sec:flight_performance_sig}. Typical \Euclid pixel has a readout noise $\sigma_{\sfont R}\approx 13 {\rm e}^{-}$ and is represented by orange lines. *Right panels:* Bias due to $\Delta f_{\rm e}$. The shaded area corresponds to the typical \Euclid dispersion of gain values. (*fig:DPU_bias*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2602.03574"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

125  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

23  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

9  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
